In [1]:
import time
import numpy as np
import pandas as pd
import geopandas
from geopandas.tools import sjoin

In short, Peromyscus maniculatus was split into 5 names circa 2019 making it a good MSW test case for tax splitting


summerize stats: 
    mammels: with, without coords
        Peromyscus with, without coords
            maniculatus with, without coords


and overview of interesting fields & their proportion of presence
lookup the dynamic properties for additional metrics (i.e., measurements)
probabilistic approach wrt morphometrics (if measurements are present)


GBIF source citation: 

GBIF.org (04 June 2021) GBIF Occurrence Download https://doi.org/10.15468/dl.phjg43 

Unless GBIF discovers citations of this download, the data file is eligible for deletion after June 4, 2022.

In [2]:
# a list of the newly split out names (Bradley et. al., 2019)
# commented out names are not accepted by MDD
name_author = {"Peromyscus arcticus" : "P. arcticus (Wagner 1845)",
                 "Peromyscus gambelii" : "P. gambelii (Baird 1857)",
                 "Peromyscus labecula" : "P. labecula (Elliot 1903)",
                 "Peromyscus maniculatus" : "P. maniculatus (Wagner 1845)",
                 "Peromyscus sonoriensis" : "P. sonoriensis (LeConte 1853)",
              }
new_names = list(name_author.keys())

In [3]:
# load in the range maps geopackage
gdf = geopandas.read_file("MDD_Rodentia_NAm_393species.gpkg")

# restrict the gdf to those ranges of interest
gdf = gdf[gdf['sciname'].isin(new_names)].copy()

# calculate area for everything
gdf.sindex # prepare spatial index

# check gdf projection
print(f"gdf is projected in: {gdf.crs}")

# examine availble gdf columns
display(gdf.columns)

/home/john/.local/lib/python3.9/site-packages/geopandas/geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


gdf is projected in: epsg:4326


Index(['sciname', 'order', 'family', 'author', 'year', 'citation',
       'rec_source', 'geometry'],
      dtype='object')

In [4]:
# load in occurence data
occ = pd.read_csv("gbif_occurrences/0294669-200613084148143/occurrence.txt", sep='\t', header=0, low_memory=False)

# clean out incorrect coords.
occ = occ[occ['decimalLongitude'] < 0]

occ.reset_index(drop=True, inplace=True)

# might be able to skip the pandas df to gdf by directly loading to geopandas.
occ = geopandas.GeoDataFrame(occ, geometry=geopandas.points_from_xy(occ['decimalLongitude'], occ['decimalLatitude']))
occ = occ.set_crs("EPSG:4326")

print(occ.shape)

(184271, 251)


#### use spatial join to for point-in-polygon checks

In [5]:
print(f"started process at: {time.ctime()}")

occ['suggested_names'] = ""
for sci in gdf['sciname'].unique():
    print(sci)
    sub = gdf[gdf['sciname']==sci] #target taxa rank
    within_points = geopandas.sjoin(occ, sub, op = 'within').index
    occ.loc[occ.index[within_points], 'suggested_names'] += f" | {name_author.get(sci, sci)}"

# clean the leading " | " from suggested_names
occ['suggested_names'] = occ['suggested_names'].str.lstrip(" | ")

# combine those with multiple suggestions for ease of visualization
occ.loc[occ['suggested_names'].str.count("\|") == 1, 'suggested_names'] = "two suggested names"
occ.loc[occ['suggested_names'].str.count("\|") > 1, 'suggested_names'] = "more than two suggested names"

# fill any empty suggestions
occ.loc[occ['suggested_names']=="", 'suggested_names'] = "out of range"

print(f"finished process at: {time.ctime()}")
occ.to_csv("peromyscus_concepts.csv", index=False)

# generalize the outcomes
x = len(occ[occ['suggested_names'].isin(name_author.values())])
print(f"ratio of occurrences resulting in one suggestion: {(x / len(occ))}")

# multiple suggested names are delimited by "|"
x = len(occ[occ['suggested_names'] == "two suggested names"])
print(f"ratio of occurrences resulting in two suggestions {(x / len(occ))}")

x = len(occ[occ['suggested_names']== "more than two suggested names"])
print(f"total number of occurrences resulting in >1 suggestions: {x}")

x = len(occ[occ['suggested_names']== "out of range"])
print(f"total number of occurrences resulting in no suggestion: {x}")
print(f"       ratio of occurrences resulting in no suggestion: {(x / len(occ))}")

started process at: Mon Jun 14 17:20:07 2021
Peromyscus arcticus
Peromyscus gambelii
Peromyscus labecula
Peromyscus maniculatus
Peromyscus sonoriensis
finished process at: Mon Jun 14 17:20:52 2021
ratio of occurrences resulting in one suggestion: 0.9017208350744285
ratio of occurrences resulting in two suggestions 0.03659284423484976
total number of occurrences resulting in >1 suggestions: 0
total number of occurrences resulting in no suggestion: 11367
       ratio of occurrences resulting in no suggestion: 0.061686320690721816


#### evaluate computation times when using buffered points

In [6]:
buffered_occ = occ.copy()

null_query = pd.isnull(buffered_occ['coordinateUncertaintyInMeters'])
print(f"Total rows with null uncertainty values: {len(buffered_occ[null_query])}")
print(f"    out of {len(buffered_occ)}")

    
print(f"started process at: {time.ctime()}")

# set nulls uncertainties to mininum of 5m buffer
buffered_occ.loc[null_query, 'coordinateUncertaintyInMeters'] = 5
# specific projection can be evaluated later, the goal here is to check computation time.
buffered_occ = buffered_occ.to_crs(epsg=3763)
buffered_occ['geometry'] = buffered_occ['geometry'].buffer(buffered_occ['coordinateUncertaintyInMeters'])
projected_gdf = gdf.copy()
projected_gdf = projected_gdf.to_crs(epsg=3763)

buffered_occ['suggested_names'] = ""
for sci in gdf['sciname'].unique():
    print(sci)
    sub = projected_gdf[projected_gdf['sciname']==sci] #target taxa rank
    within_points = geopandas.sjoin(buffered_occ, sub, op = 'within').index
    buffered_occ.loc[buffered_occ.index[within_points], 'suggested_names'] += f" | {name_author.get(sci, sci)}"

# clean the leading " | " from suggested_names
buffered_occ['suggested_names'] = buffered_occ['suggested_names'].str.lstrip(" | ")

# combine those with multiple suggestions for ease of visualization
buffered_occ.loc[buffered_occ['suggested_names'].str.count("\|") == 1, 'suggested_names'] = "two suggested names"
buffered_occ.loc[buffered_occ['suggested_names'].str.count("\|") > 1, 'suggested_names'] = "more than two suggested names"

# fill any empty suggestions
buffered_occ.loc[buffered_occ['suggested_names']=="", 'suggested_names'] = "out of range"

print(f"finished process at: {time.ctime()}")

# generalize the outcomes
x = len(buffered_occ[buffered_occ['suggested_names'].isin(name_author.values())])
print(f"ratio of occurrences resulting in one suggestion: {(x / len(buffered_occ))}")

# multiple suggested names are delimited by "|"
x = len(buffered_occ[buffered_occ['suggested_names'] == "two suggested names"])
print(f"ratio of occurrences resulting in two suggestions {(x / len(buffered_occ))}")

x = len(buffered_occ[buffered_occ['suggested_names']== "more than two suggested names"])
print(f"total number of occurrences resulting in >1 suggestions: {x}")

x = len(buffered_occ[buffered_occ['suggested_names']== "out of range"])
print(f"total number of occurrences resulting in no suggestion: {x}")
print(f"       ratio of occurrences resulting in no suggestion: {(x / len(buffered_occ))}")

# clean up the variables used to test runtimes of buffered methods
del buffered_occ
del projected_gdf

Total rows with null uncertainty values: 91247
    out of 184271
started process at: Mon Jun 14 17:22:14 2021
Peromyscus arcticus
Peromyscus gambelii
Peromyscus labecula
Peromyscus maniculatus
Peromyscus sonoriensis
finished process at: Mon Jun 14 17:37:24 2021
ratio of occurrences resulting in one suggestion: 0.8847892506145839
ratio of occurrences resulting in two suggestions 0.0363594922695378
total number of occurrences resulting in >1 suggestions: 0
total number of occurrences resulting in no suggestion: 14530
       ratio of occurrences resulting in no suggestion: 0.07885125711587825


#### evaluate computation times when using buffered ranges

Note: This test was eventually halted due to time constraints. Calculating range buffers could be done ahead of time to circumvent any added computation times during disambiguation. Keeping the section present but commented out for future reference.

In [7]:
# print(f"started process at: {time.ctime()}")

# projected_occ = occ.copy()
# # operate on a sample due to the increased computation time
# projected_occ = projected_occ.sample(5000)

# # specific projection can be evaluated later, the goal here is to check computation time.
# projected_occ = projected_occ.to_crs(epsg=3763)

# buffered_gdf = gdf.to_crs(epsg=3763)
# buffered_gdf['geometry'] = buffered_gdf['geometry'].buffer(-100) # use neg value buffer to shrink the range polygon

# buffered_occ['suggested_names'] = ""
# for sci in gdf['sciname'].unique():
#     print(sci)
#     sub = buffered_gdf[buffered_gdf['sciname']==sci] #target taxa rank
#     within_points = geopandas.sjoin(projected_occ, sub, op = 'within').index
#     projected_occ.loc[projected_occ.index[within_points], 'suggested_names'] += f" | {name_author.get(sci, sci)}"

# # clean the leading " | " from suggested_names
# projected_occ['suggested_names'] = projected_occ['suggested_names'].str.lstrip(" | ")

# # combine those with multiple suggestions for ease of visualization
# projected_occ.loc[projected_occ['suggested_names'].str.count("\|") == 1, 'suggested_names'] = "two suggested names"
# projected_occ.loc[projected_occ['suggested_names'].str.count("\|") > 1, 'suggested_names'] = "more than two suggested names"

# # fill any empty suggestions
# projected_occ.loc[projected_occ['suggested_names']=="", 'suggested_names'] = "out of range"

# print(f"finished process at: {time.ctime()}")

# # generalize the outcomes
# x = len(projected_occ[projected_occ['suggested_names'].isin(name_author.values())])
# print(f"ratio of occurrences resulting in one suggestion: {(x / len(buffered_occ))}")

# # multiple suggested names are delimited by "|"
# x = len(projected_occ[projected_occ['suggested_names'] == "two suggested names"])
# print(f"ratio of occurrences resulting in two suggestions {(x / len(buffered_occ))}")

# x = len(projected_occ[projected_occ['suggested_names']== "more than two suggested names"])
# print(f"total number of occurrences resulting in >1 suggestions: {x}")

# x = len(projected_occ[projected_occ['suggested_names']== "out of range"])
# print(f"total number of occurrences resulting in no suggestion: {x}")
# print(f"       ratio of occurrences resulting in no suggestion: {(x / len(buffered_occ))}")

# # clean up the variables used to test runtimes of buffered methods
# del projected_occ
# del buffered_gdf

#### Summarise and validate the outcomes


The goal of this section is to quantify how frequently the names we suggest align with the suggestions presented in the Appendix of Bradley et. al., 2019. To do this:
- the appendix was extracted as a CSV from the PDF
- the catalog numbers (i.e., "Museum") in the appendix were aligned for the format used in GBIF
- the records Bradley et. al. was uncertain of (i.e., "Peromyscus sp.") were omitted from the appendix 
- the gbif derived occurrence records were then inner joined with the appendix records based on the aligned
- a 0/1 boolean column is derived which identifies if the names this process suggests (i.e., 'suggested_names') are equal to the names the appendix listed.

In [8]:
occ['suggested_names'].unique().tolist()

['P. sonoriensis (LeConte 1853)',
 'P. maniculatus (Wagner 1845)',
 'out of range',
 'P. gambelii (Baird 1857)',
 'two suggested names',
 'P. labecula (Elliot 1903)',
 'P. arcticus (Wagner 1845)']

In [9]:
# The appendix table from Bradley et. al. 2019 was parsed out into the csv file below.
# parsing was done using table selection tool from: Okular version 21.04.0

Bradley_appendix = pd.read_csv("Bradley_2019_appendix.csv")
# table selection has left a single hanging whitespace in each value, so clean it column by column.
# column names were manually cleaned from csv.
for col in Bradley_appendix.columns:
    Bradley_appendix[col] = Bradley_appendix[col].str.strip()

# Bradley's Museum column should closely map to occ's "catalogNumber"
# examine samples of each to validate that.

print("Museum column from appenxis of Bradley et. al. 2019")
display(Bradley_appendix['Museum'].sample(5)) # examine "Museum" column

print("catalogNumber column from GBIF occurrence records (aka: 'occ')")
display(occ['catalogNumber'].sample(5)) # examine "catalogNumbers"

# 'Museum' column frequently includes a space between a catalog number and a museum abbreviation
# split each value in 'Museum' on white space, keep the first element as a museum abbreviation
abbreviations = Bradley_appendix['Museum'].str.split().str[0].unique()
display(abbreviations) # examine the abbreviations, and derive a simplified list

# simplified the list based on the displayed abbreviations
abbreviations = ['MSB', 'TTU', 'MVZ', 'WM', 'HGPeke', 'UAM',
                 'UWBM', 'OSM', 'CM', 'SP', 'NK', 'CRD', 'JFS',
                 'CIB', 'EAR', 'BYU', 'TK', 'Jonesboro']

Museum column from appenxis of Bradley et. al. 2019


589    MSB 90220
158     UAM20662
577    MSB 75495
554    TTU 62040
143    UAM 74970
Name: Museum, dtype: object

catalogNumber column from GBIF occurrence records (aka: 'occ')


121272                     41134
45654     RSKM_OWLPELLET_OP-4883
39622     RSKM_OWLPELLET_OP-8671
167085           MSB:Mamm:112470
92028             UAM:Mamm:51984
Name: catalogNumber, dtype: object

array(['MSB', 'TTU', 'MVZ', 'WM', 'HGPeke', 'UAM', 'UWBM', 'UAM23437',
       'UAM23438', 'UAM31104', 'UAM48129', 'UAM49643', 'UAM23667',
       'UAM43024', 'UAM23775', 'UAM20620', 'UAM20662', 'UAM20664',
       'UAM23657', 'UAM23658', 'UAM23730', 'UAM23771', 'UAM23773',
       'UAM30343', 'UAM30590', 'UAM30819', 'UAM30873', 'UAM31105',
       'UAM31106', 'UAM35318', 'UAM42578', 'UAM42579', 'UAM43469',
       'UAM44887', 'UAM50436', 'UAM52310', 'UAM60272', 'UAM74125',
       'UAM74128', 'UAM74131', 'UAM76238', 'UAM76353', 'UAM76821',
       'UAM76822', 'UAM23106', 'OSM', 'CM', 'CM109317', 'SP', 'CM109548',
       'NK', 'CRD', 'JFS', 'CLP1113', 'CLP1114', 'CLP1116', 'CLP1117',
       'CLP1118', 'CLP1119', 'CLP1120', 'CLP1121', 'CLP1122', 'CLP1123',
       'CLP1124', 'CLP1125', 'CLP1126', 'CIB', 'MSB156182', 'MSB156150',
       'MSB156183', 'MSB83399', 'MSB83409', 'MSB83419', 'MSB83417',
       'MSB156370', 'MSB156589', 'MSB56704', 'MSB56705', 'MSB56706',
       'MSB156364', 'MSB156365',

In [10]:
# iterate over each abbreviation to find which are present among occ's catalogNumber
present_abbreviations = [] # container to hold the results
for abb in abbreviations:
    qty_present = len(occ[occ['catalogNumber'].str.contains(abb)]['catalogNumber'])
    if qty_present > 0:
        present_abbreviations.append(abb)

# finally examine the mapping from Bradley's appendix to GBIF's (occ) catalogNumber
for abb in present_abbreviations:
    sample = Bradley_appendix[Bradley_appendix['Museum'].str.contains(abb)]['Museum'].sample(1).tolist()[0]
    print(f"Bradley's 'Museum' represents {abb} as: {sample}")
    
    sample = occ[occ['catalogNumber'].str.contains(abb)]['catalogNumber'].sample(1).tolist()[0]
    print(f"occ 'catalogNumber' represents {abb} as: {sample}")
    print()

# among those results, it looks like "UCM:Mamm:#####" is a false posative for "CM ######"
# remove CM from the list of present_abbreviations
present_abbreviations.remove("CM")

Bradley's 'Museum' represents MSB as: MSB 151511
occ 'catalogNumber' represents MSB as: MSB:Mamm:256706

Bradley's 'Museum' represents MVZ as: MVZ 208028
occ 'catalogNumber' represents MVZ as: MVZ:Mamm:171805

Bradley's 'Museum' represents UAM as: UAM 31107
occ 'catalogNumber' represents UAM as: UAM:Mamm:71549

Bradley's 'Museum' represents UWBM as: UWBM 75436
occ 'catalogNumber' represents UWBM as: UWBM:Mamm:49307

Bradley's 'Museum' represents CM as: CM 109407
occ 'catalogNumber' represents CM as: UCM:Mamm:20859



In [11]:
def mus_to_cat(museum_acc_number):
    """
    a function used to convert museum accession numbers extracted from the appenxid of Bradly et. al. 2019
    into the format used for GBIF's (occ) catalogNumber
    """
    return ":Mamm:".join(museum_acc_number.split())

# filter records in which Bradley_appendix is uncertain of a name
Bradley_appendix = Bradley_appendix[Bradley_appendix['sciName'] != 'Peromyscus sp.']

# filter the Bradley_appendix based on presence of a present_abbreviations in 'Museum'
abb_pattern = '|'.join(r"\b{}\b".format(x) for x in present_abbreviations)
Bradley_appendix = Bradley_appendix[Bradley_appendix['Museum'].str.contains(abb_pattern)]
# transform "Museum" col into GBIF-ish format using mus_to_cat
Bradley_appendix['catalogNumber'] = Bradley_appendix['Museum'].apply(mus_to_cat)

trimmed_Bradley_appendix = Bradley_appendix[['catalogNumber', 'Museum', 'GenBank', 'sciName']]
trimmed_occ = occ[['gbifID', 'catalogNumber', 'scientificName', 'suggested_names']]

mapped_records = trimmed_Bradley_appendix.merge(trimmed_occ,
                                                on='catalogNumber', 
                                                suffixes=('_Bradley', '_GBIF'))

mapped_records.rename({'sciName':'Bradley_name',
                       'Museum':'Bradley_museum',
                      'scientificName':'GBIF_name'}, axis=1, inplace=True)

# reverse the sci_name conversion for mapping names to Bradley
author_name = {v:k for k, v in name_author.items()}
mapped_records['suggested_names'] = mapped_records['suggested_names'].apply(lambda x: author_name.get(x,x))

# generate a "name_agreement" column for simplicity
mapped_records['name_agreement'] = mapped_records['Bradley_name'] == mapped_records['suggested_names']
mapped_records['name_agreement'] = mapped_records['name_agreement'].astype(int)

# reorder columns to preference
mapped_records = mapped_records[['catalogNumber', 'Bradley_museum', 'GenBank', 'gbifID',
                                 'GBIF_name', 'Bradley_name', 'suggested_names', 'name_agreement']]

# examine the result & shape
print(f"mapped_records shape: {mapped_records.shape}")
display(mapped_records.sample(3))
# write out the results
mapped_records.to_csv('geo_align_validation.csv', index=False)

mapped_records shape: (205, 8)


,catalogNumber,Bradley_museum,GenBank,gbifID,GBIF_name,Bradley_name,suggested_names,name_agreement
17,MVZ:Mamm:207857,MVZ 207857,EF666151,1145785190,"Peromyscus maniculatus (Wagner, 1845)",Peromyscus gambelii,out of range,0
189,MSB:Mamm:72135,MSB 72135,DQ385688,1145362964,"Peromyscus maniculatus (Wagner, 1845)",Peromyscus sonoriensis,Peromyscus sonoriensis,1
121,MSB:Mamm:74885,MSB 74885,DQ385672,1145366013,"Peromyscus maniculatus (Wagner, 1845)",Peromyscus sonoriensis,Peromyscus sonoriensis,1


In [12]:
mapped_records.groupby("suggested_names").describe()['name_agreement']

,count,mean,std,min,25%,50%,75%,max
suggested_names,,,,,,,,
Peromyscus arcticus,2.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
Peromyscus gambelii,11.0,1.00000,0.000000,1.0,1.0,1.0,1.0,1.0
Peromyscus maniculatus,8.0,0.37500,0.517549,0.0,0.0,0.0,1.0,1.0
Peromyscus sonoriensis,119.0,0.92437,0.265524,0.0,1.0,1.0,1.0,1.0
out of range,65.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
